In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet169, DenseNet121
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras.backend as K
from sklearn.metrics import roc_auc_score, cohen_kappa_score
import cv2
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import Sequence
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tqdm import tqdm
import csv
import math
from keras.callbacks import LearningRateScheduler
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/Colab Data/Weights"

In [ ]:
global_cls = ['Normal', 'Pneumothorax']
global_cls = np.array(global_cls)

In [ ]:
#X_train=np.load('/content/gdrive/My Drive/Colab Data/x_train_final(1).npy', mmap_mode='r+')
#Y_train=np.load('/content/gdrive/My Drive/Colab Data/y_train_final_classification.npy', mmap_mode='r+')
X_test=np.load('/content/gdrive/My Drive/Colab Data/X_test.npy', mmap_mode='r+')
#Y_test=np.load('/content/gdrive/My Drive/Colab Data/y_val_final_classification.npy', mmap_mode='r+')

In [ ]:
dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(256,256,3),pooling='avg')

In [ ]:
preds = Dense(2,activation='softmax')(dense_model.output)
model = Model(dense_model.input,preds)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
!ls "/content/gdrive/My Drive/Colab Data/Weights"

In [ ]:
model.load_weights('/content/gdrive/My Drive/Colab Data/Weights/DenseNet169_classifier_lr_cw.hdf5')

In [ ]:
y_pred=model.predict(X_test, verbose=1)

In [ ]:
Y_test_new=[np.argmax(i) for i in Y_test]
y_pred_new=[np.argmax(i) for i in y_pred]

In [ ]:
from sklearn.metrics import classification_report

Y_test_new=[np.argmax(i) for i in Y_test]
#y_pred_new=[np.argmax(i) for i in y_pred]
    
print(classification_report(Y_test_new, y_pred_new, target_names=global_cls))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test_new, y_pred_new)

In [ ]:
y_pred=np.array(y_pred)
df=pd.DataFrame(y_pred)
df.to_csv('/content/gdrive/My Drive/Colab Data/CSVs/testset_model6_predictions.csv', index=False)

In [ ]:
y_pred_new=[np.argmax(i) for i in y_pred]
y_pred_new=np.array(y_pred_new)
df=pd.DataFrame(y_pred_new)
df.to_csv('/content/gdrive/My Drive/Colab Data/CSVs/testset_model6_classes.csv', index=False)

In [ ]:
man_y=[]

for i in y_pred:
  if i[1]>0.05:
    man_y.append(1)
  else:
    man_y.append(0)

In [ ]:
confusion_matrix(Y_test_new, man_y)

In [ ]:
! ls '/content/gdrive/My Drive/Colab Data/CSVs'

# ENSEMBLING

## Majority Vote

In [ ]:
model1_preds=list(pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_classweights_classes.csv')['0'])
model2_preds=list(pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_augment_wocw_classes.csv')['0'])
model3_preds=list(pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_classweights_2_classes.csv')['0'])
model4_preds=list(pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_classweights_augment_classes.csv')['0'])
model5_preds=list(pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_classweights_classes.csv')['0'])
model6_preds=list(pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet169_classifier_lr_cw_classes.csv')['0'])

In [ ]:
y=[]
for i in range(0, len(model1_preds)):
  count_of_ones=model1_preds[i]+model2_preds[i]+model3_preds[i]+model4_preds[i]+model5_preds[i]+model6_preds[i]
  count_of_zeros=6-count_of_ones
  if count_of_ones>count_of_zeros:
    y.append(1)
  else:
    y.append(0)

In [ ]:
y_pred_new=y

## Average Voting


In [ ]:
#model1_preds=list(pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_classweights_predictions.csv')['0'])
model2_preds=pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_augment_wocw_predictions.csv')
model3_preds=pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_classweights_2_predictions.csv')
model4_preds=pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_classweights_augment_predictions.csv')
model5_preds=pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet121_classifier_reducelr_classweights_predictions.csv')
model6_preds=pd.read_csv('/content/gdrive/My Drive/Colab Data/CSVs/DenseNet169_classifier_lr_cw_predictions.csv')

In [ ]:
model3_preds

In [ ]:
y=[]
for i in range(0, len(model2_preds)):
  total_0=model2_preds['0'][i]+model3_preds['0'][i]+model4_preds['0'][i]+model5_preds['0'][i]+model6_preds['0'][i]
  total_1=model2_preds['1'][i]+model3_preds['1'][i]+model4_preds['1'][i]+model5_preds['1'][i]+model6_preds['1'][i]
  print (total_0, total_1)
  if total_1>total_0:
    y.append(1)
  else:
    y.append(0)

In [ ]:
y_pred_new=y